<a href="https://colab.research.google.com/github/aenderson/aenderson-Projeto-de-pesquisa-2020-Inteligencia-artificial-no-futebol/blob/main/Intelig%C3%AAncia_artificial_no_futebol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas

In [ ]:
#Abrindo arquivo
filepath = 'E0.csv'
df = pandas.read_csv(filepath)
print(df)

In [ ]:
#Transformando letras em numeros (H = 1, A = 0, D = 0)

for i in range(len(df)):
  if df.iloc[i,7] == 'H':
    df.iloc[i,7] = 1
  if df.iloc[i,7] == 'A':
    df.iloc[i,7] = 0
  if df.iloc[i,7] == 'D':
    df.iloc[i,7] = 0

In [ ]:
extract = [12,13,14,15,16,17,18,19,20,21,22,23,24,26,27,29,30,32,33,35,36,38,39,41,7]
df = df.iloc[:, extract]
print(df)

In [ ]:
#normalizando os dados entre 0 e 1
for i in range(len(df.columns)):
  print((df.iloc[:, i].max()))
  max = (df.iloc[:, i].max())
  if max < 10:
    df.iloc[:, i] /= 10
  elif max < 100:
    df.iloc[:, i] /= 100
  elif max < 1000:
    df.iloc[:, i] /= 1000
  else:
    df.iloc[:, i] /= 10000
print(df)

In [ ]:
# embaralhando os dados
df = df.sample(frac=1)
print(df)

In [ ]:
# separando dados de treino e teste
for i in range(len(df)):
  if df.iloc[i, 24] == 0.1:
    df.iloc[i, 24] = 1

treino = df.iloc[: -24]
test = df.iloc[-20 :]
print(treino)
print(test)

In [ ]:
# dividindo os dados entre input e output
treino_input = treino.iloc[:, :-1]
treino_output = treino.iloc[:, 24:]
print(treino_input)
print(treino_output)

test_input = test.iloc[:, :-1]
test_output = test.iloc[:, 24:]
print(test_input)
print(test_output)

In [ ]:
import torch
import numpy as np

In [ ]:
class Net(torch.nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Net, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.fc1 = torch.nn.Linear(self.input_size, self.hidden_size)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(self.hidden_size, 1)
        self.sigmoid = torch.nn.Sigmoid() 
    def forward(self, x):
        hidden = self.fc1(x)
        relu = self.relu(hidden)
        output = self.fc2(relu)
        output = self.sigmoid(output)
        return output

In [ ]:
# convertendo para tensors
treino_input = treino_input.astype(np.float)
treino_input = torch.FloatTensor(treino_input.values)
treino_output = treino_output.astype(np.float)
treino_output = torch.FloatTensor(treino_output.values)
test_input = test_input.astype(np.float)
test_input = torch.FloatTensor(test_input.values)
test_output = test_output.astype(np.float)
test_output = torch.FloatTensor(test_output.values)
print(type(treino_input))
print(treino_input.size())
print(type(treino_output))
print(treino_output.size())
print(type(test_input))
print(test_input.size())
print(type(test_output))
print(test_output.size())

In [ ]:
input_size = treino_input.size()[1]
hidden_size = 30
model = Net(input_size, hidden_size)
criterion = torch.nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.8, momentum=0.3)

In [ ]:
model.eval()
y_pred = model(test_input)
before_train = criterion(y_pred.squeeze(), test_output.squeeze())
print('Perda antes do treinamento' , before_train.item())

In [ ]:
model.train()
epochs = 45000
errors = []
for epoch in range(epochs):
    optimizer.zero_grad()
    # Forward pass
    y_pred = model(treino_input)
    # Compute Loss
    loss = criterion(y_pred.squeeze(), treino_output.squeeze())
    errors.append(loss.item())
    if epoch % 1000 == 0:
      print('Epoch {}: train loss: {}'.format(epoch, loss.item()))
    # Backward pass
    loss.backward()
    optimizer.step()

In [ ]:
model.eval()
y_pred = model(test_input)
after_train = criterion(y_pred.squeeze(), test_output.squeeze())
print('Perda depois do treinamento' , after_train.item())

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
def plotcharts(errors):
    errors = np.array(errors)
    plt.figure(figsize=(12, 5))
    graf02 = plt.subplot(1, 2, 1) # nrows, ncols, index
    graf02.set_title('Errors')
    plt.plot(errors, '-')
    plt.xlabel('Epochs')
    graf03 = plt.subplot(1, 2, 2)
    graf03.set_title('Tests')
    a = plt.plot(test_output.numpy(), 'yo', label='Real')
    plt.setp(a, markersize=10)
    a = plt.plot(y_pred.detach().numpy(), 'b+', label='Predicted')
    plt.setp(a, markersize=10)
    plt.legend(loc=7)
    plt.show()
plotcharts(errors)